#### c'est pas encore bon : il faut encore trouver un moyen d'éviter d'avoir des dates proches 

In [1]:
import pandas as pd
import numpy as np

source_data = '../collect/binance_history/ETHUSDT_binance_daily.csv'
df = pd.read_csv(source_data)
df['date'] = pd.to_datetime(df['Time']/1000, unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
df = df[pd.to_datetime(df['date']) < pd.to_datetime('2021-05-18')]

In [8]:
df[df['date'] == '2021-04-18 00:00:00']

,Time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore,date
281,1618704000000,2319.93,2341.91,1900.0,2235.32,4275573.298,1618790399999,9.242083e+09,3573690,2031438.001,4.391973e+09,0,2021-04-18 00:00:00


In [3]:
df_drawdowns = pd.DataFrame(columns=["drawdown", "date_debut", "high", "date_fin", "low"])

In [10]:
# Initialisation d'une liste pour collecter les données
drawdowns_data = []
n = len(df)

# Calcul des drawdowns
for i in range(n):
    print(i , '/' ,n)
    minimum_global = 1000000
    date_fin = None
    high = df['high'][i]
    for j in range(i, n):
        minimum = df['low'].iloc[i:j+1].min()
        drawdown_max = minimum / high - 1 
        if minimum < minimum_global :
            minimum_global = minimum
            date_fin = df['date'][j]
    if minimum_global / high - 1 < -0.03:
        diff = minimum_global / high - 1
        drawdowns_data.append({'drawdown': diff, 'date_debut': df['date'][i], 'high': df['high'][i], 'date_fin': date_fin, 'low': minimum_global})

drawdowns = pd.DataFrame(drawdowns_data)
drawdowns.set_index('date_debut', inplace=True)
drawdowns.index = pd.to_datetime(drawdowns.index)
drawdowns['date_fin'] = pd.to_datetime(drawdowns['date_fin'])

0 / 310
1 / 310
2 / 310
3 / 310
4 / 310
5 / 310
6 / 310
7 / 310
8 / 310
9 / 310
10 / 310
11 / 310
12 / 310
13 / 310
14 / 310
15 / 310
16 / 310
17 / 310
18 / 310
19 / 310
20 / 310
21 / 310
22 / 310
23 / 310
24 / 310
25 / 310
26 / 310
27 / 310
28 / 310
29 / 310
30 / 310
31 / 310
32 / 310
33 / 310
34 / 310
35 / 310
36 / 310
37 / 310
38 / 310
39 / 310
40 / 310
41 / 310
42 / 310
43 / 310
44 / 310
45 / 310
46 / 310
47 / 310
48 / 310
49 / 310
50 / 310
51 / 310
52 / 310
53 / 310
54 / 310
55 / 310
56 / 310
57 / 310
58 / 310
59 / 310
60 / 310
61 / 310
62 / 310
63 / 310
64 / 310
65 / 310
66 / 310
67 / 310
68 / 310
69 / 310
70 / 310
71 / 310
72 / 310
73 / 310
74 / 310
75 / 310
76 / 310
77 / 310
78 / 310
79 / 310
80 / 310
81 / 310
82 / 310
83 / 310
84 / 310
85 / 310
86 / 310
87 / 310
88 / 310
89 / 310
90 / 310
91 / 310
92 / 310
93 / 310
94 / 310
95 / 310
96 / 310
97 / 310
98 / 310
99 / 310
100 / 310
101 / 310
102 / 310
103 / 310
104 / 310
105 / 310
106 / 310
107 / 310
108 / 310
109 / 310
110 / 310


In [20]:
drawdowns

,drawdown,high,date_fin,low
date_debut,,,,
2020-07-02,-0.039786,231.99,2020-07-02 00:00:00,222.76
2020-07-04,-0.032543,230.77,2020-07-05 00:00:00,223.26
2020-07-06,-0.063676,242.32,2020-07-06 00:00:00,226.89
2020-07-07,-0.057647,243.90,2020-07-16 00:00:00,229.84
2020-07-08,-0.076948,249.00,2020-07-16 00:00:00,229.84
...,...,...,...,...
2021-05-13,-0.225805,4036.45,2021-05-17 00:00:00,3125.00
2021-05-14,-0.251497,4175.00,2021-05-17 00:00:00,3125.00
2021-05-15,-0.244660,4137.21,2021-05-17 00:00:00,3125.00


In [ ]:
# il faut run plusieurs fois et uncomment la deuxième ligne

In [32]:
# Initialiser un DataFrame pour stocker les lignes filtrées
to_keep = []
#drawdowns = drawdowns_filtered

# Itérer sur chaque ligne du DataFrame
for i, row in drawdowns.iterrows():
    # Trouver les dates dans la plage de 5 jours avant et après
    start_date = i - pd.Timedelta(days=10)
    end_date = i + pd.Timedelta(days=10)
    
    # Filtrer le DataFrame pour cette plage de dates
    df_temp = drawdowns.loc[start_date:end_date]
    
    # Trouver l'index de la ligne avec le 'drawdown' minimal dans cette plage
    min_drawdown_idx = df_temp['drawdown'].idxmin()
    
    # Ajouter l'index de la ligne avec le 'drawdown' minimal à la liste
    if min_drawdown_idx not in to_keep:
        to_keep.append(min_drawdown_idx)

# Concaténer toutes les lignes filtrées dans un nouveau DataFrame
drawdowns_filtered = drawdowns.loc[to_keep]
drawdowns_filtered

,drawdown,high,date_fin,low
date_debut,,,,
2020-07-08,-0.076948,249.00,2020-07-16,229.84
2020-08-02,-0.278358,418.49,2020-08-02,302.00
2020-08-17,-0.315457,449.00,2020-09-05,307.36
2020-09-02,-0.362905,482.44,2020-09-05,307.36
2020-09-17,-0.207349,394.07,2020-09-23,312.36
2020-10-22,-0.123015,421.90,2020-11-03,370.00
2020-11-07,-0.098012,469.74,2020-11-07,423.70
2020-11-24,-0.231378,624.65,2020-11-26,480.12
2020-12-17,-0.193733,678.20,2020-12-23,546.81


In [29]:
drawdowns_filtered.to_csv('drawdown.csv')

In [35]:
drawdowns_filtered.mean()

drawdown                        -0.238009
high                          1176.505714
date_fin    2020-12-01 18:51:25.714285824
low                            860.912143
dtype: object

In [36]:
diffs = drawdowns_filtered.index.to_series().diff() # Calculer les différences entre les dates consécutives
average_diff = diffs.mean() # Calculer la moyenne de ces différences
average_diff

Timedelta('23 days 16:36:55.384615384')

In [ ]:
drawdowns_sorted = drawdowns.sort_values(by='drawdown', ascending=True)

In [ ]:
drawdowns_sorted

In [ ]:
#drawdowns_sorted_filtered = drawdowns_sorted.loc[drawdowns_sorted.groupby('date_fin_jour')['drawdown'].idxmin()]
drawdowns_sorted_filtered = drawdowns_sorted.loc[drawdowns_sorted.groupby('date_debut_jour')['drawdown'].idxmin()]

In [ ]:
drawdowns_sorted_filtered